<a href="https://colab.research.google.com/github/parvathymr21/ml-algo/blob/main/Activelearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.metrics import accuracy_score,mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
    

# 4. Apply active learning techniques on Mammographic mass data set.
(a) Discuss the technique used for active learning.
(b) Discuss the experimental settings and results.


In [ ]:
data2 = pd.read_csv('mammographic_masses.data',header=None)
data2.head()
data2[0].unique()
d = data2.copy()
data2.shape
#data2[0].replace(to_replace='?' , value=" ")
#data2.isna().sum()

(961, 6)

In [ ]:
data2
for i in range((data2.shape[1])) :
    data2.iloc[:,i] = data2.iloc[:,i].replace('?', 
            np.mean(pd.to_numeric(data2.iloc[:,i], errors='coerce')))
data2

,0,1,2,3,4,5
0,5,67,3,5,3,1
1,4,43,1,1,2.910734,1
2,5,58,4,5,3,1
3,4,28,1,1,3,0
4,5,74,1,5,2.910734,1
...,...,...,...,...,...,...
956,4,47,2,1,3,0
957,4,56,4,5,3,1
958,4,64,4,5,3,0
959,5,66,4,5,3,1


In [ ]:
D = data2.iloc[:,:-1]

D = D.astype(float)
value = []
for i in range(5):
    value.append(np.median(D[i]))
value
#D.columns[5]
for j in D.columns:
    data2.iloc[:,j]=data2.iloc[:,j].replace(to_replace='?' , value = np.median(D[j]))
#data2[1]=data2[1].replace(to_replace='?' , value=np.median(X[1]))
#data2[2]=data2[2].replace(to_replace='?' , value=np.median(X[2]))
data2 = data2.astype(int)
data2

,0,1,2,3,4,5
0,5,67,3,5,3,1
1,4,43,1,1,2,1
2,5,58,4,5,3,1
3,4,28,1,1,3,0
4,5,74,1,5,2,1
...,...,...,...,...,...,...
956,4,47,2,1,3,0
957,4,56,4,5,3,1
958,4,64,4,5,3,0
959,5,66,4,5,3,1


In [ ]:
X = data2.iloc[:,:-1]
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X = sc.fit_transform(X)
X= pd.DataFrame(X)
y = data2.iloc[:,-1]


x_train, x_pool, y_train, y_pool = train_test_split(X, y, train_size = 0.1,random_state =42)

x_train_lab, x_test_unlab, y_train_lab, y_test_unlab = train_test_split(x_pool, y_pool, test_size=0.3, random_state=42)


print(x_train.shape[0],'\n',y_train.shape[0],'\n',x_train_lab.shape[0])
classifier1 = LogisticRegression()
classifier1.fit(x_train, y_train)
y_probab = classifier1.predict_proba(x_test_unlab)
y_prob = []
for i in range((x_test_unlab.shape[0])):
    y_prob.append(np.max(y_probab[i]))
c = pd.DataFrame({'prob':y_prob,'x_test_index':x_test_unlab.index})
p = 0.47 # range of uncertanity 0.47 to 0.53
uncrt_pt_ind = []
print(x_train_lab.shape[0])
for i in range(x_test_unlab.shape[0]):
    if((y_prob[i]) >= p and y_prob[i] <= 1-p):
        uncrt_pt_ind.append(c.iloc[i,-1])
#print(c)
        
   # x_test_unlab = np.delete(x_test_unlab,i,0)
print(uncrt_pt_ind)
y_train = pd.DataFrame(y_train)
y_test_unlab = pd.DataFrame(y_test_unlab)
y_train_lab = pd.DataFrame(y_train_lab)

for i in uncrt_pt_ind:
    
    #index_pd = x_test_unlab.iloc[:,:i]
    #print(x_test_unlab.loc[i,:])
    x_train = x_train.append(x_test_unlab.loc[i,:])
    y_train = y_train.append(y_test_unlab.loc[i])
    x_test_unlab = x_test_unlab.drop(i,axis=0)
    y_test_unlab = y_test_unlab.drop(i,axis=0)
classifier2 = LogisticRegression()
classifier2.fit(x_train, y_train)
acc=(classifier2.score(x_test_unlab, y_test_unlab))
print('Accuracy by active learning:',acc)


96 
 96 
 605
605
[936, 163, 110, 489, 410, 790, 940, 284, 429, 480, 570]
Accuracy by active learning: 0.8232931726907631


C:\Users\RIJUS OACHIRA\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
